# Loading most recent data from postgres table

## Installing needed packages and connections

In [9]:
# Install libraries
!pip install psycopg2-binary
!pip install requests
!pip install sqlalchemy --upgrade
!pip install dask --upgrade


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load data and analytics using Dask

In [13]:
import os
import dask.dataframe as dd
from api_query import api_query_call

df = api_query_call("SELECT * FROM uniswap_table")


In [11]:
# Trim dataframe
df = df[["block_timestamp_time","transaction_to_address","transaction_txfrom_address","close_price","basecurrency_symbol" \
        ,"quotecurrency_symbol"]]

df[0:10].head()

,block_timestamp_time,transaction_to_address,transaction_txfrom_address,close_price,basecurrency_symbol,quotecurrency_symbol
0,2023-05-04 19:19:11,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2,30531.225998786256,ETH,HEX
1,2023-05-04 19:19:11,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2,0.0000327533522577755,HEX,ETH
2,2023-05-04 18:53:23,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0xc11099d3cd9686158db0a40e88267e7b1740958c,0.0005315458950839691,DAI,ETH
3,2023-05-04 18:53:23,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0xc11099d3cd9686158db0a40e88267e7b1740958c,1881.3050937812784,ETH,DAI
4,2023-05-04 17:24:59,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2,1882.5397559820076,ETH,USDC


In [12]:
# Select only the rows where basecurrency is USDC
USDC_Transactions = df[df['basecurrency_symbol'] == 'USDC']
USDC_Transactions.head()

,block_timestamp_time,transaction_to_address,transaction_txfrom_address,close_price,basecurrency_symbol,quotecurrency_symbol
5,2023-05-04 17:24:59,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,0xd7e1236c08731c3632519dcd1a581bfe6876a3b2,0.0005311972811317125,USDC,ETH
17,2023-05-04 16:14:59,0xa88800cd213da5ae406ce248380802bd53b47647,0x9108813f22637385228a1c621c1904bbbc50dc25,0.0005344938794312031,USDC,ETH
23,2023-05-04 13:42:11,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x4432e64624f4c64633466655de3d5132ad407343,0.000529984844454519,USDC,ETH
42,2023-05-04 00:24:47,0xe4000004000bd8006e00720000d27d1fa000d43e,0x524cfae2daab901234c842f3a17235902b0f01f9,0.0005252414200862029,USDC,ETH
47,2023-05-03 22:39:47,0x0000000000a84d1a9b0063a910315c7ffa9cd248,0x5e3d1448a0365c3b1cd08908d410bfccca449c45,0.0005249748063127298,USDC,ETH


In [8]:
# We can see the highest trading wallets
counts = df[['transaction_to_address', 'transaction_txfrom_address']].stack().value_counts()

# Print the wallet with the highest transactions
print(counts)



0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0    142
0xd7e1236c08731c3632519dcd1a581bfe6876a3b2    128
0xdef1c0ded9bec7f1a1670819833240f027b25eff    116
0x0eae044f00b0af300500f090ea00027097d03000     76
0x0000000000007f150bd6f54c40a34d7c3d5e9f56     66
                                             ... 
0xa2a372fabda69b660295574f121fa83c388d476a      2
0x54b0bf5f9ae860e130413a2d6c5f9ddbef8f0e5c      2
0x991be4464807af4b20a1159a3445ac5dbd5674ca      2
0xd63fd5eb9d485fd5352343bea816ccc74b4f7323      2
0xc6c7565644ea1893ad29182f7b6961aab7edfed0      2
Length: 242, dtype: int64


In [25]:
# Group the rows by object and sum up the cost
subset = df.iloc[:, :6]
print(subset.head())

   tradeindex block_timestamp_time  block_height exchange_fullname protocol  \
0          15  2023-05-03 22:39:47      17183209           Uniswap  Uniswap   
1          15  2023-05-03 22:39:47      17183209           Uniswap  Uniswap   
2          11  2023-05-03 22:35:35      17183188           Uniswap  Uniswap   
3          11  2023-05-03 22:35:35      17183188           Uniswap  Uniswap   
4          10  2023-05-03 22:35:35      17183188           Uniswap  Uniswap   

  timeinterval_minute  
0 2023-05-03 22:39:00  
1 2023-05-03 22:39:00  
2 2023-05-03 22:35:00  
3 2023-05-03 22:35:00  
4 2023-05-03 22:35:00  


In [23]:
# Group the rows by object and sum up the cost
import os
import dask.dataframe as dd

           baseamount         quoteamount  trades               quoteprice  \
0   2597.662901948303   2610.064734550981       1       1904.8533148165884   
1   2610.064734550981   2597.662901948303       1    0.0005249748063127298   
2  251.06246686521044  250.03942847251892       1       1888.0772403321316   
3  250.03942847251892  251.06246686521044       1    0.0005296393487716053   
4   252.8014772247965  251.06246686521044       1  0.000031954331131291345   

             maximum_price            minimum_price               open_price  \
0       1904.8533148165884       1904.8533148165884       1904.8533148165884   
1    0.0005249748063127298    0.0005249748063127298    0.0005249748063127298   
2       1888.0772403321316       1888.0772403321316       1888.0772403321316   
3    0.0005296393487716053    0.0005296393487716053    0.0005296393487716053   
4  0.000031954331131291345  0.000031954331131291345  0.000031954331131291345   

               close_price basecurrency_symbol  
0

In [4]:

from api_query import api_query_call
import pandas as pd

import dask.dataframe as dd

# Load dataframe from uniswap_table
query = "SELECT * FROM uniswap_table"
df_pandas = api_query_call(query)

# Clear previous query_results
# api_query_call("DROP TABLE query_calc")

# Use DASK to partition pandas dataframe for faster processing
df = dd.from_pandas(df_pandas, npartitions=10)  

# We can see the highest trading wallets
counts = df['transaction_to_address'].value_counts()
pandas_frame = counts.compute().to_frame(name="high_trader")
